In [1]:
from fairseq.data.data_utils import load_indexed_dataset
from fairseq.models.bart import BARTModel
from fairseq.data import Dictionary
import torch.nn.functional as F 

import torch
import torch.nn as nn

import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import os
# os.environ['MKL_THREADING_LAYER'] = 'GNU'
# os.system('CUDA_LAUNCH_BLOCKING=0')


/home/gayane/miniconda3/envs/rk/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-07-14 09:38:40 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-07-14 09:38:40 | INFO | numexpr.utils | NumExpr defaulting to 4 threads.


In [2]:
dataset_name = 'BBBP'
# dataset = dataset_name if dataset_name in set(["BBBP", "BACE", "HIV"]) else f"{dataset_name}_{args.subtask}"
dataset = dataset_name

store_path = "/home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data"
model = f"{store_path}/{dataset}/processed"

with open('/home/gayane/BartLM/captum/fairseq/scripts/datasets.json') as f:
    datasets_json = json.load(f)
dataset_js = datasets_json[dataset]
task_type = dataset_js['type']

if task_type == "regression":
    mi = dataset_js['minimum']
    ma = dataset_js['maximum']

os.system(f"mkdir -p {store_path}/{dataset}/")
os.system(f"mkdir -p {store_path}/{dataset}/processed/")
os.system(f"mkdir -p {store_path}/{dataset}/processed/input0/")
os.system(f"mkdir -p {store_path}/{dataset}/processed/label/")

warmup = 163
totNumUpdate = 1020
lr = '3e-5'
chkpt_path = '/mnt/good/gayane/data/chkpt/BBBP_bs_16_dropout_0.1_lr_3e-5_totalNum_1020_warmup_163_noise_type_uniform_r3f_lambda_1.0/checkpoint_best.pt'
# chkpt_path = f"/mnt/good/gayane/data/chkpt/{dataset}_bs_16_lr_{lr}_totalNum_{totNumUpdate}_warmup_{warmup}/checkpoint_last.pt"
print(chkpt_path)  # BACE_bs_16_lr_3e-5_totalNum_1135_warmup_181/ in test 
bart = BARTModel.from_pretrained(model,  checkpoint_file = chkpt_path, 
                                 bpe="sentencepiece",
                                 sentencepiece_model="/home/gayane/BartLM/Bart/chemical/tokenizer/chem.model")
bart.eval()
bart.cuda(device=1)


2022-07-14 09:38:47 | INFO | fairseq.file_utils | loading archive file /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed


/mnt/good/gayane/data/chkpt/BBBP_bs_16_dropout_0.1_lr_3e-5_totalNum_1020_warmup_163_noise_type_uniform_r3f_lambda_1.0/checkpoint_best.pt


2022-07-14 09:38:55 | INFO | fairseq.tasks.sentence_prediction | [input] dictionary: 1026 types
2022-07-14 09:38:55 | INFO | fairseq.tasks.sentence_prediction | [label] dictionary: 9 types
2022-07-14 09:39:00 | INFO | fairseq.models.bart.model | Registering classification head: sentence_classification_head


BARTHubInterface(
  (models): ModuleList(
    (0): BARTModel(
      (encoder): TransformerEncoderBase(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(1026, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(130, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): Fairse

In [4]:
# bart.cuda(device=1)

In [58]:
data_type = 'test'

input_dict = Dictionary.load(f"{store_path}/{dataset}/processed/input0/dict.txt")
smiles = list(load_indexed_dataset(
    f"{store_path}/{dataset}/processed/input0/{data_type}", input_dict))


test_label_path = f"{store_path}/{dataset}/processed/label/{data_type}"

if task_type == 'classification':
    
    target_dict = Dictionary.load(f"{store_path}/{dataset}/processed/label/dict.txt")
    targets = list(load_indexed_dataset(test_label_path, target_dict))
elif task_type == 'regression':
    with open(f'{test_label_path}.label') as f:
        lines = f.readlines()
        targets = [float(x.strip()) for x in lines]
if task_type == 'classification':
    if len(dataset_js["class_index"])>1:
        target_dict = list()
        targets_list = list()
        for i in range(len(dataset_js["class_index"])):
            target_dict.append(Dictionary.load(f"{store_path}/{dataset}_{i}/processed/label/dict.txt"))
            targets_list.append(list(load_indexed_dataset(test_label_path[i], target_dict[i])))
        
    else: 
        target_dict = Dictionary.load(f"{store_path}/{dataset}/processed/label/dict.txt")
        targets = list(load_indexed_dataset(test_label_path, target_dict))
elif task_type == 'regression':
    with open(f'{test_label_path}.label') as f:
        lines = f.readlines()
        targets = [float(x.strip()) for x in lines]


2022-07-14 09:57:55 | INFO | fairseq.data.data_utils | loaded 204 examples from: /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed/input0/test
2022-07-14 09:57:55 | INFO | fairseq.data.data_utils | loaded 204 examples from: /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed/label/test
2022-07-14 09:57:55 | INFO | fairseq.data.data_utils | loaded 204 examples from: /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed/label/test


In [59]:
for i, (smile, target) in tqdm(list(enumerate(zip(smiles, targets)))):
    smile = torch.cat((torch.cat((torch.tensor([0]), smile[:126])), torch.tensor([2])))

100%|██████████| 204/204 [00:00<00:00, 43621.62it/s]


In [60]:
bart.decode(bart.encode("CCC.[H+]"))

'CCC.[H+]'

In [61]:
smi = []
for sm in smiles:
    smi.append(bart.decode(torch.tensor(sm)))

smi[:3]

/tmp/ipykernel_718867/2256499831.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  smi.append(bart.decode(torch.tensor(sm)))


['CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O',
 'FC(F)(F)[C@]1(OC(=O)Nc2ccc(Cl)cc12)C#CC3CC3',
 'CN1C[C@@H](C[C@H]2[C@H]1Cc3c[nH]c4cccc2c34)C(=O)N[C@]5(C)O[C@@]6(O)[C@@H]7CCCN7C(=O)[C@H](Cc8ccccc8)N6C5=O']

In [62]:
bart.decode(bart.encode('CC․OC(=O[C@H](CCc1ccccc1)N[C@@H(C)C(=O)N2CCC[C@H]2C(O)=O'))

'CC.OC(=O[C@H](CCc1ccccc1)N[C@@H(C)C(=O)N2CCC[C@H]2C(O)=O'

In [63]:
_input = pd.read_csv(f"/home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/raw/{data_type}.input", names=['SMILES'], header=None)

In [65]:
_input

,SMILES
0,CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[...
1,FC(F)(F)[C@]1(OC(=O)Nc2ccc(Cl)cc12)C#CC3CC3
2,CN1C[C@@H](C[C@H]2[C@H]1Cc3c[nH]c4cccc2c34)C(=...
3,CN1CCC[C@H]1c2cccnc2
4,CN1CCC(=CC1)c2ccccc2
...,...
199,Nc1nnc(c(N)n1)c2cccc(Cl)c2Cl
200,COc1ccc(Cl)cc1C(=O)NCCc2ccc(cc2)[S](=O)(=O)NC(...
201,Cn1c2CCC(Cn3ccnc3C)C(=O)c2c4ccccc14
202,CN(C)[C@H]1[C@@H]2C[C@H]3C(=C(O)c4c(O)cccc4[C@...


In [66]:
e = bart.encode('CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O')
output = bart.predict('sentence_classification_head', e )
# target = target[0].item()
output[0][1].exp().item()

0.9972198009490967

In [67]:
e = bart.encode('CCOC(=)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O')
output = bart.predict('sentence_classification_head', e )
# target = target[0].item()
output, output[0][1].exp().item()


(tensor([[-5.8841e+00, -2.7873e-03]], device='cuda:1',
        grad_fn=<LogSoftmaxBackward0>),
 0.9972166419029236)

In [68]:
e = bart.encode('CCOC()[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O')
output = bart.predict('sentence_classification_head', e )
# target = target[0].item()
output[0][1].exp().item()

0.9972227215766907

In [69]:
# li = valid_input["SMILES"].to_list()
li = _input["SMILES"].to_list()

In [70]:
false_smile = []
false_recons = []
for i in range(len(li)):
    if smi[i] != li[i]:
        false_smile.append(li[i])
        false_recons.append(smi[i])


In [71]:
d = {"Original SMILES": false_smile, "Reconstracted SMILES": false_recons}
df_bbbp = pd.DataFrame(d)
# df_valid_bbbp = pd.DataFrame(d) 

In [72]:
len(li), df_bbbp.shape[0], len(li) - df_bbbp.shape[0]

(204, 17, 187)

In [73]:
df_bbbp

,Original SMILES,Reconstracted SMILES
0,C[S](O)(=O)=O.Oc1ccc2[nH]cc(CCCCN3CCC(=CC3)c4c...,C[S](O)(=O)=O<unk>Oc1ccc2[nH]cc(CCCCN3CCC(=CC3...
1,CN1C2CCC1CC(C2)OC(c3ccccc3)c4ccccc4.O[S](O)(=O)=O,CN1C2CCC1CC(C2)OC(c3ccccc3)c4ccccc4<unk>O[S](O...
2,OC(=O)\C=C/C(O)=O.OC(=O)\C=C/C(O)=O.C1CCN(CC1)...,OC(=O)\C=C/C(O)=O<unk>OC(=O)\C=C/C(O)=O<unk>C1...
3,CN[C@H](CC(C)C)C(=O)NC1[C@H](O)c2ccc(Oc3cc4cc(...,CN[C@H](CC(C)C)C(=O)NC1[C@H](O)c2ccc(Oc3cc4cc(...
4,[Ca++].NC1=NC(=O)C2=C(NCC(CNc3ccc(cc3)C(=O)N[C...,[C<unk>++]<unk>NC1=NC(=O)C2=C(NCC(CNc3ccc(cc3)...
5,[Br-].C[N+]1(C)CCCC(C1)OC(=O)C(O)(c2ccccc2)c3c...,[Br-]<unk>C[N+]1(C)CCCC(C1)OC(=O)C(O)(c2ccccc2...
6,[Br-].CN(C)C(=O)Oc1ccc[n+](C)c1,[Br-]<unk>CN(C)C(=O)Oc1ccc[n+](C)c1
7,[Br-].C[N+]1(C)CCC(C1)OC(=O)C(O)(C2CCCC2)c3ccccc3,[Br-]<unk>C[N+]1(C)CCC(C1)OC(=O)C(O)(C2CCCC2)c...
8,[Br-].C[N+]12CCC(CC1)C(C2)OC(=O)C(O)(c3ccccc3)...,[Br-]<unk>C[N+]12CCC(CC1)C(C2)OC(=O)C(O)(c3ccc...
9,O.COC1CN(CCCOc2ccc(F)cc2)CCC1NC(=O)c3cc(Cl)c(N...,O<unk>COC1CN(CCCOc2ccc(F)cc2)CCC1NC(=O)c3cc(Cl...


In [37]:
df_valid_bbbp.values[-1][0], df_bbbp.values[-1][0]

('[Cl].CC[C@H]1CN2CCc3cc(OC)c(OC)cc3[C@@H]2C[C@@H]1C[C@H]4NCCc5cc(OC)c(OC)cc45',
 '[Cl].CC[C@H]1CN2CCc3cc(OC)c(OC)cc3[C@@H]2C[C@@H]1C[C@H]4NCCc5cc(OC)c(OC)cc45')

In [1]:
e = bart.decode(bart.encode('CCOC(=)<mask>(CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O'))
# print(e)
# bart.eval()
# bart.fill_mask(masked_inputs = ['CCOC<mask>'],  topk=3, beam=10, match_source_len=False)


NameError: name 'bart' is not defined

In [83]:
tokens = bart.encode("CCOC(=O)(CCc1ccccc1)N[C@@H](C)C(=O)N2CCC[C@H]2C(O)=O")
last_layer_features = bart.extract_features(tokens)

In [80]:
last_layer_features.size()

torch.Size([1, 40, 1024])

In [2]:
from fairseq.data.data_utils import load_indexed_dataset
from fairseq.models.bart import BARTModel
from fairseq.data import Dictionary
import torch.nn.functional as F 

import torch
import torch.nn as nn

import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import os
# os.environ['MKL_THREADING_LAYER'] = 'GNU'
# os.system('CUDA_LAUNCH_BLOCKING=0')

dataset_name = 'BBBP'
# dataset = dataset_name if dataset_name in set(["BBBP", "BACE", "HIV"]) else f"{dataset_name}_{args.subtask}"
dataset = dataset_name

store_path = "/home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data"
model = f"{store_path}/{dataset}/processed"

with open('/home/gayane/BartLM/captum/fairseq/scripts/datasets.json') as f:
    datasets_json = json.load(f)
dataset_js = datasets_json[dataset]
task_type = dataset_js['type']

if task_type == "regression":
    mi = dataset_js['minimum']
    ma = dataset_js['maximum']

os.system(f"mkdir -p {store_path}/{dataset}/")
os.system(f"mkdir -p {store_path}/{dataset}/processed/")
os.system(f"mkdir -p {store_path}/{dataset}/processed/input0/")
os.system(f"mkdir -p {store_path}/{dataset}/processed/label/")

warmup = 163
totNumUpdate = 1020
lr = '3e-5'
chkpt_path = '/mnt/good/gayane/data/chkpt/BBBP_bs_16_dropout_0.1_lr_3e-5_totalNum_1020_warmup_163_noise_type_uniform_r3f_lambda_1.0/checkpoint_best.pt'
# chkpt_path = f"/mnt/good/gayane/data/chkpt/{dataset}_bs_16_lr_{lr}_totalNum_{totNumUpdate}_warmup_{warmup}/checkpoint_last.pt"
print(chkpt_path)  # BACE_bs_16_lr_3e-5_totalNum_1135_warmup_181/ in test 
bart = BARTModel.from_pretrained(model,  checkpoint_file = chkpt_path, 
                                 bpe="sentencepiece",
                                 sentencepiece_model="/home/gayane/BartLM/Bart/chemical/tokenizer/chem.model")
bart.eval()
bart.cuda(device=0)


data_type = 'valid'

input_dict = Dictionary.load(f"{store_path}/{dataset}/processed/input0/dict.txt")
smiles = list(load_indexed_dataset(
    f"{store_path}/{dataset}/processed/input0/{data_type}", input_dict))


test_label_path = f"{store_path}/{dataset}/processed/label/{data_type}"

if task_type == 'classification':
    
    target_dict = Dictionary.load(f"{store_path}/{dataset}/processed/label/dict.txt")
    targets = list(load_indexed_dataset(test_label_path, target_dict))
elif task_type == 'regression':
    with open(f'{test_label_path}.label') as f:
        lines = f.readlines()
        targets = [float(x.strip()) for x in lines]
if task_type == 'classification':
    if len(dataset_js["class_index"])>1:
        target_dict = list()
        targets_list = list()
        for i in range(len(dataset_js["class_index"])):
            target_dict.append(Dictionary.load(f"{store_path}/{dataset}_{i}/processed/label/dict.txt"))
            targets_list.append(list(load_indexed_dataset(test_label_path[i], target_dict[i])))
        
    else: 
        target_dict = Dictionary.load(f"{store_path}/{dataset}/processed/label/dict.txt")
        targets = list(load_indexed_dataset(test_label_path, target_dict))
elif task_type == 'regression':
    with open(f'{test_label_path}.label') as f:
        lines = f.readlines()
        targets = [float(x.strip()) for x in lines]

y_pred = []
y = []
sm = []
for i, (smile, target) in tqdm(list(enumerate(zip(smiles, targets)))):
    smile = torch.cat((torch.cat((torch.tensor([0]), smile[:126])), torch.tensor([2])))  
    if task_type =="classification":
        output = bart.predict('sentence_classification_head', smile)
        target = target[0].item()
        y_pred.append(output[0][1].exp().item())
        y.append(target - 4)
        sm.append(bart.decode(smile))
        
    elif task_type == "regression":
        output = bart.predict('sentence_classification_head', smile, return_logits=True)
        y_pred.append(output[0][0].item())
        y.append(target)
        sm.append(bart.decode(smile))





_input = pd.read_csv(f"/home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/raw/{data_type}.input", names=['SMILES'], header=None)
print(len(_input))

li = _input["SMILES"].to_list()
l = _input["SMILES"].to_list()

smi = []
for sml in range(len(li)):
    if "(" in li[sml]:
        li[sml] = li[sml].replace('(', "", 1)
for s in li:
    smi.append(bart.decode(bart.encode(s)))

2022-07-15 16:46:21 | INFO | fairseq.file_utils | loading archive file /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed


/mnt/good/gayane/data/chkpt/BBBP_bs_16_dropout_0.1_lr_3e-5_totalNum_1020_warmup_163_noise_type_uniform_r3f_lambda_1.0/checkpoint_best.pt


2022-07-15 16:46:23 | INFO | fairseq.tasks.sentence_prediction | [input] dictionary: 1026 types
2022-07-15 16:46:23 | INFO | fairseq.tasks.sentence_prediction | [label] dictionary: 9 types
2022-07-15 16:46:28 | INFO | fairseq.models.bart.model | Registering classification head: sentence_classification_head
2022-07-15 16:46:29 | INFO | fairseq.data.data_utils | loaded 204 examples from: /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed/input0/valid
2022-07-15 16:46:29 | INFO | fairseq.data.data_utils | loaded 204 examples from: /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed/label/valid
2022-07-15 16:46:29 | INFO | fairseq.data.data_utils | loaded 204 examples from: /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed/label/valid
100%|██████████| 204/204 [00:05<00:00, 39.27it/s]

204


In [5]:
l[0], li[0], smi[0]

('[H+].C2=C1C(OC(=NC1=CC=C2Cl)NCC)(C3=CC=CC=C3)C.[Cl-]',
 '[H+].C2=C1COC(=NC1=CC=C2Cl)NCC)(C3=CC=CC=C3)C.[Cl-]',
 '[H+].C2=C1COC(=NC1=CC=C2Cl)NCC)(C3=CC=CC=C3)C.[Cl-]')

In [4]:

li = _input["SMILES"].to_list()
l = _input["SMILES"].to_list()

smi = []
for sml in range(len(li)):
    if "(" in li[sml]:
        li[sml] = li[sml].replace('(', "", 1)
for s in li:
    smi.append(bart.decode(bart.encode(s)))

In [10]:
queries = '[H+].C2=C1C(OC(=NC1=CC=C2Cl)NCC)(C3=CC=CC=C3)C.[Cl-]'
seq =     '[H+].C2=C1COC(=NC1=CC=C2Cl)NCC)(C3=CC=CC=C3)C.[Cl-]'
import regex as re
for q in queries:
    m = re.search(r'(%s){e<=1}'%q, seq)
    a = m if m else 'nomatch'
    print(a)

error: unterminated character set at position 9

In [12]:
def substrings(s1, s2):
    final = [s1[i:b+1] for i in range(len(s1)) for b in range(len(s1))]


    return [i for i in final if i in s1 and i in s2 and len(i) > 1]

In [13]:
substrings(queries, seq)

['[H',
 '[H+',
 '[H+]',
 '[H+].',
 '[H+].C',
 '[H+].C2',
 '[H+].C2=',
 '[H+].C2=C',
 '[H+].C2=C1',
 '[H+].C2=C1C',
 'H+',
 'H+]',
 'H+].',
 'H+].C',
 'H+].C2',
 'H+].C2=',
 'H+].C2=C',
 'H+].C2=C1',
 'H+].C2=C1C',
 '+]',
 '+].',
 '+].C',
 '+].C2',
 '+].C2=',
 '+].C2=C',
 '+].C2=C1',
 '+].C2=C1C',
 '].',
 '].C',
 '].C2',
 '].C2=',
 '].C2=C',
 '].C2=C1',
 '].C2=C1C',
 '.C',
 '.C2',
 '.C2=',
 '.C2=C',
 '.C2=C1',
 '.C2=C1C',
 'C2',
 'C2=',
 'C2=C',
 'C2=C1',
 'C2=C1C',
 '2=',
 '2=C',
 '2=C1',
 '2=C1C',
 '=C',
 '=C1',
 '=C1C',
 'C1',
 'C1C',
 '1C',
 'C(',
 'OC',
 'OC(',
 'OC(=',
 'OC(=N',
 'OC(=NC',
 'OC(=NC1',
 'OC(=NC1=',
 'OC(=NC1=C',
 'OC(=NC1=CC',
 'OC(=NC1=CC=',
 'OC(=NC1=CC=C',
 'OC(=NC1=CC=C2',
 'OC(=NC1=CC=C2C',
 'OC(=NC1=CC=C2Cl',
 'OC(=NC1=CC=C2Cl)',
 'OC(=NC1=CC=C2Cl)N',
 'OC(=NC1=CC=C2Cl)NC',
 'OC(=NC1=CC=C2Cl)NCC',
 'OC(=NC1=CC=C2Cl)NCC)',
 'OC(=NC1=CC=C2Cl)NCC)(',
 'OC(=NC1=CC=C2Cl)NCC)(C',
 'OC(=NC1=CC=C2Cl)NCC)(C3',
 'OC(=NC1=CC=C2Cl)NCC)(C3=',
 'OC(=NC1=CC=C2Cl)NCC)(C3=C',

In [1]:
from fairseq.data.data_utils import load_indexed_dataset
from fairseq.models.bart import BARTModel
from fairseq.data import Dictionary
import torch.nn.functional as F 

import torch
import torch.nn as nn

import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import os


dataset_name = 'BBBP'
# dataset = dataset_name if dataset_name in set(["BBBP", "BACE", "HIV"]) else f"{dataset_name}_{args.subtask}"
dataset = dataset_name

store_path = "/home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data"
model = f"{store_path}/{dataset}/processed/input0"

with open('/home/gayane/BartLM/captum/fairseq/scripts/datasets.json') as f:
    datasets_json = json.load(f)
dataset_js = datasets_json[dataset]
task_type = dataset_js['type']

if task_type == "regression":
    mi = dataset_js['minimum']
    ma = dataset_js['maximum']

os.system(f"mkdir -p {store_path}/{dataset}/")
os.system(f"mkdir -p {store_path}/{dataset}/processed/")
os.system(f"mkdir -p {store_path}/{dataset}/processed/input0/")
os.system(f"mkdir -p {store_path}/{dataset}/processed/label/")

data_type = "valid"

warmup = 163
totNumUpdate = 1020
lr = '3e-5'
chkpt_path = '/mnt/good/gayane/data/chkpt/BBBP_bs_16_dropout_0.1_lr_3e-5_totalNum_1020_warmup_163_noise_type_uniform_r3f_lambda_1.0/checkpoint_best.pt'
# chkpt_path = f"/mnt/good/gayane/data/chkpt/{dataset}_bs_16_lr_{lr}_totalNum_{totNumUpdate}_warmup_{warmup}/checkpoint_last.pt"
print(chkpt_path)  # BACE_bs_16_lr_3e-5_totalNum_1135_warmup_181/ in test 
# bart = BARTModel.from_pretrained(model,  checkpoint_file = chkpt_path, 
bart = BARTModel.from_pretrained(model,  checkpoint_file = '/home/gayane/BartLM/checkpoints/checkpoint_last.pt', 
                                 bpe="sentencepiece",
                                 sentencepiece_model="/home/gayane/BartLM/Bart/chemical/tokenizer/chem.model")
bart.eval()
bart.cuda(device=1)


input_dict = Dictionary.load(f"{store_path}/{dataset}/processed/input0/dict.txt")
smiles = list(load_indexed_dataset(
    f"{store_path}/{dataset}/processed/input0/{data_type}", input_dict))


test_label_path = f"{store_path}/{dataset}/processed/label/{data_type}"

if task_type == 'classification':
    
    target_dict = Dictionary.load(f"{store_path}/{dataset}/processed/label/dict.txt")
    targets = list(load_indexed_dataset(test_label_path, target_dict))
elif task_type == 'regression':
    with open(f'{test_label_path}.label') as f:
        lines = f.readlines()
        targets = [float(x.strip()) for x in lines]
if task_type == 'classification':
    if len(dataset_js["class_index"])>1:
        target_dict = list()
        targets_list = list()
        for i in range(len(dataset_js["class_index"])):
            target_dict.append(Dictionary.load(f"{store_path}/{dataset}_{i}/processed/label/dict.txt"))
            targets_list.append(list(load_indexed_dataset(test_label_path[i], target_dict[i])))
        
    else: 
        target_dict = Dictionary.load(f"{store_path}/{dataset}/processed/label/dict.txt")
        targets = list(load_indexed_dataset(test_label_path, target_dict))
elif task_type == 'regression':
    with open(f'{test_label_path}.label') as f:
        lines = f.readlines()
        targets = [float(x.strip()) for x in lines]

import re
smi = []
for sm in smiles:
    smi.append(bart.decode(torch.tensor(sm)))

_input = pd.read_csv(f"""/home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/raw/{data_type}.input""", names=['SMILES'], header=None)
li = _input["SMILES"].to_list()
# print(li[:2])




/home/gayane/miniconda3/envs/rk/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-07-28 09:17:01 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-07-28 09:17:02 | INFO | numexpr.utils | NumExpr defaulting to 4 threads.
2022-07-28 09:17:02 | INFO | fairseq.file_utils | loading archive file /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed/input0


/mnt/good/gayane/data/chkpt/BBBP_bs_16_dropout_0.1_lr_3e-5_totalNum_1020_warmup_163_noise_type_uniform_r3f_lambda_1.0/checkpoint_best.pt


2022-07-28 09:17:10 | INFO | fairseq.tasks.denoising | dictionary: 1025 types
2022-07-28 09:17:17 | INFO | fairseq.data.data_utils | loaded 204 examples from: /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed/input0/valid
2022-07-28 09:17:17 | INFO | fairseq.data.data_utils | loaded 204 examples from: /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed/label/valid
2022-07-28 09:17:17 | INFO | fairseq.data.data_utils | loaded 204 examples from: /home/gayane/BartLM/Bart/chemical/checkpoints/evaluation_data/BBBP/processed/label/valid
/tmp/ipykernel_323375/3928296069.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  smi.append(bart.decode(torch.tensor(sm)))


In [2]:
l = _input["SMILES"].to_list()


In [3]:
# input_dict.

In [30]:
_input["SMILES"][0]

'[H+].C2=C1C(OC(=NC1=CC=C2Cl)NCC)(C3=CC=CC=C3)C.[Cl-]'

In [5]:
# l = _input["SMILES"].to_list()
# for sml in range(len(li)):
#     # li[sml] = li[sml][:122] + "<mask>"
#     if "(" in li[sml]:
#         li[sml] = li[sml][:122].replace('(', "<mask>", 1)


# print(li)
inputs = ['C(=O)(OC(C)(C)C)CCCc1ccc(<mask>1)N(CCCl)CCCl', '▁C [ C <mask> ]( C c 1 ccc ( O ) c ( O ) c 1 ) ( N N ) C ( O )= O'] #li 
results = bart.fill_mask(masked_inputs = inputs,  topk=2, match_source_len=True)
assert len(inputs) == len(results), "different sizes?"

# d = {"Original SMILES": l, "masked SMILES": li, "filled SMILES": results}
# df = pd.DataFrame(d)


In [33]:
bart.encode('[H+].C2=C1C(OC(=NC1=CC=C2Cl)NCC)(C3=CC=CC=C3)C.[Cl-]')

tensor([  0,  63,  11,  10, 357,   3,   4,  93,   4,   6,   4,   7,   8,   4,
         12,  26,  79,   4,   4,  29,   4,  13,  39,   5,  26,   4,   5,   7,
          4, 168,   4,   4,  29,   4,   4,  29,   4,  35,   4,   3,  11,  39,
        156,   2])

In [31]:
bart.decode(bart.encode("[H+].CC1C(OC(=NC1=CC=C2Cl)NCC)(C3=CC=CC=C3)C.[Cl-]"))

'[H+]<unk>CC1C(OC(=NC1=CC=C2Cl)NCC)(C3=CC=CC=C3)C<unk>[Cl-]'

In [36]:
bart.decode(torch.tensor([39]))

'Cl'

In [13]:
bart.decode(bart.encode("[H+].CC1C(OC(=NC1=CC=C2Cl)NCC)(C3=CC=CC=C3)C.[Cl-]"))

'[H+]<unk>CC1C(OC(=NC1=CC=C2Cl)NCC)(C3=CC=CC=C3)C<unk>[Cl-]'

'2='

In [ ]:
tensor([ 0, 20, 12,  8,  5,  7,  8,  4,  7,  4,  5,  7,  4,  5,  4,  5,  4,  4,
         4,  9,  6, 25,  7,  3,  6,  5, 14,  7,  4,  4, 39,  5,  4,  4, 39,  2])

In [40]:
# bart.encode("r")

tensor([   0,   63, 1024,    2])

In [33]:
results

[[('C[C@@H](CNC(=O)OC(C)(C)C)CCCc1ccc(F)cc1 O=C',
   tensor(-0.1258, device='cuda:0')),
  ('C[C@H](CNC(=O)OC(C)(C)C)CCCc1ccc(F)cc1 O=C',
   tensor(-0.1264, device='cuda:0'))],
 [('C[C@H](N)[C@@H](O)c1ccccc1 c1ccc(C(F)(F)F)c(C(F)(F)C(F)',
   tensor(-0.2962, device='cuda:0')),
  ('C[C@H](N)[C@@H](O)c1ccccc1 c1ccc(C(F)(F)F)c(C(F)(F)F)c(',
   tensor(-0.3008, device='cuda:0'))]]

In [22]:
results

[[('CCOP(=O)C(C)(C)C Cc1cc(C)c(C)c(C(=O)O)c(C)c1C(=O)O C(=O)CCCCCCCCCCC CCCOCC',
   tensor(-0.5145, device='cuda:0')),
  ('CCOP(=O)C(C)(C)C Cc1cc(C)c(C)c(C(=O)O)c(C)c1C(=O)O C(=O)CCCCCCCCCCC C1',
   tensor(-0.5182, device='cuda:0'))],
 [('C[C@H](N)[C@@H](O)c1ccccc1 c1ccc(C(F)(F)F)c(C(F)(F)C(F)',
   tensor(-0.2961, device='cuda:0')),
  ('C[C@H](N)[C@@H](O)c1ccccc1 c1ccc(C(F)(F)F)c(C(F)(F)C(F)(',
   tensor(-inf, device='cuda:0'))]]

In [44]:
list(input_dict)
